# Prepare environment

You must prepare your environment __if you have not done so in the previous exercises__ in order to do this homework. Follow the steps below.

-----
1. Run the next cell, and verify that you are logged as you, and not as someone else

In [ ]:
echo "You are ${HADOOP_USER_NAME:-nobody}"
echo "You are ${JUPYTERHUB_USER:-nobody}"

-----
2. Run the code below and verify the existance of your database. Execute step 3 if you have a database.\
Otherwise go to step 6, if it shows an empty list as shown below:

```
    +-----------------+
    |  database_name  |
    +-----------------+
    +-----------------+
```

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "show databases like '${JUPYTERHUB_USER:-nobody}'"

-----
3. Review the content of your database if you have one.

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "show tables in ${JUPYTERHUB_USER:-nobody};"

-----
4. Drop your database after having reviewed its content in step 3, and __you are ok losing its content__.

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "drop database if exists ${JUPYTERHUB_USER:-nobody} cascade;"

-----
5. Verify that you the database is gone

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "show databases like '${JUPYTERHUB_USER:-nobody}';"

-----
6. Run the remaining cells to reconstruct your hive folder and reconfigure ACL permissions on HDFS

In [ ]:
hdfs dfs -ls /user/${JUPYTERHUB_USER:-nobody}

In [ ]:
hdfs dfs -rm -r -f -skipTrash /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -rm -r -f -skipTrash /user/${usJUPYTERHUB_USER:-nobody}/.Trash

In [ ]:
hdfs dfs -mkdir -p                                /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl    -m group::r-x                /user/${JUPYTERHUB_USER:-nobody}
hdfs dfs -setfacl    -m other::---                /user/${JUPYTERHUB_USER:-nobody}
hdfs dfs -setfacl    -m default:group::r-x        /user/${JUPYTERHUB_USER:-nobody}
hdfs dfs -setfacl    -m default:other::---        /user/${JUPYTERHUB_USER:-nobody}
hdfs dfs -setfacl -R -m group::r-x                /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl -R -m other::---                /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl -R -m default:group::r-x        /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl -R -m default:other::---        /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl    -m user:hive:rwx             /user/${JUPYTERHUB_USER:-nobody}/hive
hdfs dfs -setfacl    -m default:user:hive:rwx     /user/${JUPYTERHUB_USER:-nobody}/hive

-----
7. Recreate the __external__ tables `sbb_orc` and `sbb_05_11_2018` which you will need in the homework.

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "
create database if not exists ${JUPYTERHUB_USER:-nobody} location '/user/${JUPYTERHUB_USER:-nobody}/hive';
drop table if exists ${JUPYTERHUB_USER:-nobody}.sbb_orc;
create external table ${JUPYTERHUB_USER:-nobody}.sbb_orc(
        BETRIEBSTAG string,
        FAHRT_BEZEICHNER string,
        BETREIBER_ID string,
        BETREIBER_ABK string,
        BETREIBER_NAME string,
        PRODUKT_ID string,
        LINIEN_ID string,
        LINIEN_TEXT string,
        UMLAUF_ID string,
        VERKEHRSMITTEL_TEXT string,
        ZUSATZFAHRT_TF string,
        FAELLT_AUS_TF string,
        BPUIC string,
        HALTESTELLEN_NAME string,
        ANKUNFTSZEIT string,
        AN_PROGNOSE string,
        AN_PROGNOSE_STATUS string,
        ABFAHRTSZEIT string,
        AB_PROGNOSE string,
        AB_PROGNOSE_STATUS string,
        DURCHFAHRT_TF string
    )
    row format delimited fields terminated by ';'
    stored as ORC
    location '/data/sbb/orc/istdaten'
    tblproperties ('skip.header.line.count'='1');
    select count(*) from ${JUPYTERHUB_USER:-nobody}.sbb_orc;
    "

In [ ]:
beeline -n "${JUPYTERHUB_USER:-nobody}" -p "" -e "   
create external table if not exists ${JUPYTERHUB_USER:-nobody}.sbb_05_11_2018
    stored as orc
    as 
        select *
        from ${JUPYTERHUB_USER:-nobody}.sbb_orc
        where BETRIEBSTAG = '05.11.2018'
"